<a href="https://colab.research.google.com/github/Hamdan-Naeem/Particle-Tracking/blob/main/portfolio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cv2
import matplotlib.pyplot as plt
import os
import seaborn; seaborn.set_style("whitegrid")
from tqdm import tqdm
import shutil
import numpy as np
from IPython.display import HTML

if os.path.isdir("frames"):
    shutil.rmtree("frames")
os.mkdir("frames")

if os.path.isfile("processed_video.mp4"):
    os.remove("processed_video.mp4")

video = cv2.VideoCapture('newdrop(1).mp4')

def process_and_save_image(frame, frame_number):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Greyscale allows the image/main object to be picked up easier since it is now black and white
    #greyscale_img = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    greyscale_img = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

    # The blurring degree is used to enhance circle detection by making other images/objects smoother.
    blurring_degree = (15, 15)
    blurred_img = cv2.blur(greyscale_img, blurring_degree)


    # Threshold value
    # This turns the image into black-white: pixel intensities lower than the threshold value become black.
    # Pixel intensities greater than the threshold value become white.
    # This allows the ball to be detected with heavy constrast, since it is black (our object) on a white background
    threshold = 200
    status, thresholded_img = cv2.threshold(blurred_img, threshold, 255, cv2.THRESH_BINARY)


    # The following parameters allow the algorithm to detect the ball using specific measurements
    # This limits the number of objects that can be detected, while singling out the ball

    circles = cv2.HoughCircles(
        thresholded_img,
        cv2.HOUGH_GRADIENT, 1,
        minDist = 40,               # Minimum distance between the detected circles
        param1 = 100,               # How strong object edges must be
        param2 = 10,                # How perfectly circular the objects must be
        minRadius = 30,             # Minimum acceptable circle radius (in pixels)
        maxRadius = 50,             # Maximum acceptable circle radius

    )

    # Plotting the processed images as compiled frames into graphs

    fig, axes = plt.subplots(2, 3, figsize = (15, 15))

    axes[0, 0].set(title = "Original")
    axes[0, 0].imshow(frame)

    axes[0, 1].set(title = "Greyscaled")
    axes[0, 1].imshow(greyscale_img)

    axes[1, 0].set(title = "Blurred")
    axes[1, 0].imshow(blurred_img)

    axes[1, 1].set(title = "Thresholded - Circle Detection")
    axes[1, 1].imshow(thresholded_img)

    if circles is not None:
        for circ in circles[0]:
            x, y, r = circ
            axes[1, 1].add_artist(plt.Circle((x, y), r, fill = False, color = "red"))

            frame_numbers.append(frame_number)
            xpositions.append(frame.shape[1] - x)
            ypositions.append(frame.shape[0] - y)
            radii.append(r)


# Additional graphs to plot the trajectory of the moving ball, frame by frame

    axes[0, 2].set(title = "X-Position", xlabel = "Frame Number", ylabel = "X (Pixels)",
                   xlim = [0, frame.shape[1]])
    axes[0, 2].scatter(frame_numbers, xpositions, s = radii, c = radii)

    axes[1, 2].set(title = "Y-Position", xlabel = "Frame Number", ylabel = "Y (Pixels)",
                   ylim = [0, frame.shape[0]])
    axes[1, 2].scatter(frame_numbers, ypositions, s = radii, c = radii)

    # This saves an image of the processed figure. These individual images will be compiled into the video

    fig.savefig(f"frames/frame{frame_number:0>4}.png")
    plt.close(fig)

frame_numbers = []
xpositions = []
ypositions = []
radii = []

# Read in `num_frames` frames and process them; save each resulting figure separately
num_frames = 50
for i in tqdm(range(num_frames)):
    status, frame = video.read()
    if status is False:
        break

    process_and_save_image(frame, i)

# Stitching a final video from all the processed frame
!ffmpeg -framerate 10 -i frames/frame%04d.png -c:v libx264 -r 30 -pix_fmt yuv420p -hide_banner -loglevel error processed_video.mp4

# Final video will be automatically uploaded in the files section
show_video("processed_video.mp4")


  0%|          | 0/50 [00:00<?, ?it/s]


In [ ]:
# Restitution Coefficient Calculation
# A scale factor of 90.0 cm = 1300 pixels
# v^2 = u^2 + 2as - calculation of the velocity before collision

s_1 = 90 * 0.01                  # Displacement undergone by ball from starting point to end point
Initial_velocity = 0             # Ball is being dropped
g = 9.81                         # Acceleration due to gravity

v_1 = (Initial_velocity + 2*g*s_1)**0.5
print("Final velocity before collision = ", v_1)

# Calculating velcoity after collision
Final_velocity = 0                        # Ball reaches zero velocity after converting all
                                          # kinetic energy to gravitational potential energy
s_2 = 90 * (800/1300) * 0.01              # Displacement undergone by ball from starting point to end point after rebound
v_2 = Final_velocity + (2*g*s_2)**0.5     # Velocity after collision
Final_velocity = 0
print("Velocity after collision = ", v_2)

# Restituation Coefficient
e = v_2/v_1
print("Restituation coefficient, e = ", e )








Final velocity before collision =  4.2021423107743505
Velocity after collision =  3.2964316371588143
Restituation coefficient, e =  0.7844645405527363
